<a href="https://colab.research.google.com/github/Hars99/DL-Project/blob/main/DL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install roboflow opencv-python matplotlib numpy tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [ ]:
from roboflow import Roboflow

# Initialize Roboflow
rf = Roboflow(api_key="5dfuIOYNMJxRcngWFoiP")  # Replace with your actual Roboflow API key
workspace = rf.workspace("dl-ljbpi")
project = workspace.project("augmented_dataset-cckeg")
version = project.version(1)  # Version 2 as per your dataset

# Export dataset in YOLOv8 format
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Augmented_dataset-1 in yolov8:: 100%|██████████| 4664/4664 [00:00<00:00, 7996.48it/s]


In [ ]:
import os
import shutil

# Define paths
image_dir = "/content/Augmented_dataset-1/train/images"
label_dir = "/content/Augmented_dataset-1/train/labels"
output_dir = "/content/dataset/train"

os.makedirs(os.path.join(output_dir, "normal"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "degenerative"), exist_ok=True)

for label_file in os.listdir(label_dir):
    label_path = os.path.join(label_dir, label_file)
    image_path = os.path.join(image_dir, label_file.replace(".txt", ".jpg"))

    if not os.path.exists(image_path):
        continue

    with open(label_path, 'r') as f:
        lines = f.readlines()

    count_class_1 = sum(1 for line in lines if line.startswith("1 "))

    if count_class_1 >= 2:
        dest = os.path.join(output_dir, "degenerative", os.path.basename(image_path))
    else:
        dest = os.path.join(output_dir, "normal", os.path.basename(image_path))

    shutil.copy(image_path, dest)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 1910 images belonging to 2 classes.
Found 381 images belonging to 2 classes.


DenseNet121


In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7551 - loss: 0.5601

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.


60/60 ━━━━━━━━━━━━━━━━━━━━ 266s 2s/step - accuracy: 0.7557 - loss: 0.5584 - val_accuracy: 0.7165 - val_loss: 2.0976
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 279ms/step - accuracy: 0.8642 - loss: 0.3207 - val_accuracy: 0.8478 - val_loss: 0.9774
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 283ms/step - accuracy: 0.9207 - loss: 0.2254 - val_accuracy: 0.7795 - val_loss: 3.1245
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 282ms/step - accuracy: 0.9381 - loss: 0.1640 - val_accuracy: 0.5144 - val_loss: 1.9607
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 289ms/step - accuracy: 0.9322 - loss: 0.1909 - val_accuracy: 0.5669 - val_loss: 1.5807
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 18s 290ms/step - accuracy: 0.9574 - loss: 0.1231 - val_accuracy: 0.8425 - val_loss: 0.3958
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 287ms/step - accuracy: 0.9680 - loss: 0.0831 - val_accuracy: 0.6483 - val_loss: 2.0329
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 18s 291ms/step - accuracy: 0.9712 - loss: 0.0839 - val_accuracy: 0.8688 

In [ ]:
model.save("/content/densenet_spine_model.h5")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = "/content/Augmented_dataset-1/test/images/S0029_09_png_jpg.rf.e924c1d4c8cb065f92a3572a524b9727.jpg"
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) / 255.

pred = model.predict(x)[0][0]

if pred > 0.5:
    print("Degenerative Spine")
else:
    print("Normal Spine")

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
Normal Spine


Preprocessing the valid set images

In [ ]:
import os
import shutil

# Define paths
valid_dir = "/content/Augmented_dataset-1/valid"
image_dir = os.path.join(valid_dir, "images")
label_dir = os.path.join(valid_dir, "labels")

# Output directories for reorganized data
output_dir = os.path.join(valid_dir, "reorganized")
os.makedirs(os.path.join(output_dir, "normal"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "degenerative"), exist_ok=True)

# Process each image and its corresponding label
for label_file in os.listdir(label_dir):
    if not label_file.endswith(".txt"):
        continue

    # Full paths
    label_path = os.path.join(label_dir, label_file)
    image_path = os.path.join(image_dir, label_file.replace(".txt", ".jpg"))

    # Check if image exists
    if not os.path.exists(image_path):
        print(f"Warning: Image not found for {label_file}")
        continue

    # Read label file
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Count occurrences of class 1 (degenerative)
    count_class_1 = sum(1 for line in lines if line.startswith("1 "))

    # Assign class label
    if count_class_1 >= 2:
        dest_folder = os.path.join(output_dir, "degenerative")
    else:
        dest_folder = os.path.join(output_dir, "normal")

    # Copy image to appropriate folder
    shutil.copy(image_path, os.path.join(dest_folder, os.path.basename(image_path)))

print("Dataset reorganization complete.")

Dataset reorganization complete.


In [ ]:
normal_count = len(os.listdir(os.path.join(output_dir, "normal")))
degenerative_count = len(os.listdir(os.path.join(output_dir, "degenerative")))

print(f"Normal: {normal_count} images")
print(f"Degenerative: {degenerative_count} images")

Normal: 92 images
Degenerative: 185 images


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    output_dir,  # Use the reorganized directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 277 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve
import numpy as np

# Load the saved model
model = load_model('/content/densenet_spine_model.h5')

# Define test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/Augmented_dataset-1/valid/reorganized',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Predict probabilities
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator).ravel()

# Find optimal threshold based on F1 score
precision, recall, thresholds = precision_recall_curve(y_true, y_pred_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal Threshold (F1-based): {optimal_threshold:.4f}")

# Apply optimal threshold
y_pred = (y_pred_prob > optimal_threshold).astype(int)

# Classification report
print(classification_report(y_true, y_pred))

# ROC-AUC Score
roc_auc = roc_auc_score(y_true, y_pred_prob)
print(f"ROC-AUC Score: {roc_auc:.2f}")


Found 277 images belonging to 2 classes.


Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.


9/9 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step
Optimal Threshold (F1-based): 0.0543
              precision    recall  f1-score   support

           0       0.90      0.91      0.91       185
           1       0.82      0.79      0.81        92

    accuracy                           0.87       277
   macro avg       0.86      0.85      0.86       277
weighted avg       0.87      0.87      0.87       277

ROC-AUC Score: 0.93


ResNet

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = valid_datagen.flow_from_directory(
    '/content/Augmented_dataset-1/valid/reorganized',  # Ensure this points to reorganized folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 2326 images belonging to 2 classes.
Found 277 images belonging to 2 classes.


In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Define data generators with stronger augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Load train and validation data
train_generator = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = valid_datagen.flow_from_directory(
    '/content/Augmented_dataset-1/valid/reorganized',  # Ensure this points to reorganized folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load pre-trained ResNet50 without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)  # Fewer units with L2 regularization
x = Dropout(0.7)(x)  # Higher dropout rate
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),  # More aggressive early stopping
    ModelCheckpoint("best_resnet_model.h5", save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks
)

# Unfreeze all layers for fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Recompile with lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training
history_fine = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=callbacks
)

Found 2326 images belonging to 2 classes.
Found 277 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step - accuracy: 0.6667 - loss: 1.3103

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


73/73 ━━━━━━━━━━━━━━━━━━━━ 53s 583ms/step - accuracy: 0.6670 - loss: 1.3056 - val_accuracy: 0.6679 - val_loss: 0.6923
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step - accuracy: 0.7093 - loss: 0.6629

73/73 ━━━━━━━━━━━━━━━━━━━━ 34s 466ms/step - accuracy: 0.7094 - loss: 0.6629 - val_accuracy: 0.6679 - val_loss: 0.6602
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step - accuracy: 0.7145 - loss: 0.6413

73/73 ━━━━━━━━━━━━━━━━━━━━ 35s 478ms/step - accuracy: 0.7145 - loss: 0.6414 - val_accuracy: 0.6679 - val_loss: 0.6473
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step - accuracy: 0.7146 - loss: 0.6467

73/73 ━━━━━━━━━━━━━━━━━━━━ 35s 481ms/step - accuracy: 0.7146 - loss: 0.6467 - val_accuracy: 0.6679 - val_loss: 0.6443
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step - accuracy: 0.7129 - loss: 0.6332

73/73 ━━━━━━━━━━━━━━━━━━━━ 33s 455ms/step - accuracy: 0.7129 - loss: 0.6332 - val_accuracy: 0.6679 - val_loss: 0.6362
Epoch 6/10
60/73 ━━━━━━━━━━━━━━━━━━━━ 5s 443ms/step - accuracy: 0.7139 - loss: 0.6227

KeyboardInterrupt: 

##ResNet50


In [13]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Define data generators with stronger augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Load train and validation data
train_generator = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = valid_datagen.flow_from_directory(
    '/content/Augmented_dataset-1/valid/reorganized',  # Ensure this points to reorganized folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load pre-trained ResNet50 without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)  # Fewer units with L2 regularization
x = Dropout(0.7)(x)  # Higher dropout rate
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),  # More aggressive early stopping
    ModelCheckpoint("best_resnet_model.h5", save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks
)

# Unfreeze all layers for fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Recompile with lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training
history_fine = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=callbacks
)

Found 2326 images belonging to 2 classes.
Found 277 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step - accuracy: 0.6291 - loss: 1.5588

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


73/73 ━━━━━━━━━━━━━━━━━━━━ 87s 596ms/step - accuracy: 0.6298 - loss: 1.5522 - val_accuracy: 0.6679 - val_loss: 0.7180
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step - accuracy: 0.7066 - loss: 0.7091

...

73/73 ━━━━━━━━━━━━━━━━━━━━ 33s 450ms/step - accuracy: 0.7007 - loss: 0.6384 - val_accuracy: 0.6679 - val_loss: 0.6374
Epoch 9/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step - accuracy: 0.7247 - loss: 0.6183

73/73 ━━━━━━━━━━━━━━━━━━━━ 34s 462ms/step - accuracy: 0.7246 - loss: 0.6183 - val_accuracy: 0.6679 - val_loss: 0.6321
Epoch 10/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step - accuracy: 0.7082 - loss: 0.6137

73/73 ━━━━━━━━━━━━━━━━━━━━ 34s 463ms/step - accuracy: 0.7083 - loss: 0.6137 - val_accuracy: 0.6679 - val_loss: 0.6251
Epoch 1/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 122s 898ms/step - accuracy: 0.7241 - loss: 0.6103 - val_accuracy: 0.6679 - val_loss: 0.6619
Epoch 2/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 39s 532ms/step - accuracy: 0.7037 - loss: 0.5841 - val_accuracy: 0.6679 - val_loss: 0.6633
Epoch 3/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 39s 534ms/step - accuracy: 0.7071 - loss: 0.5549 - val_accuracy: 0.6679 - val_loss: 0.6259
Epoch 4/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 517ms/step - accuracy: 0.7181 - loss: 0.5142

73/73 ━━━━━━━━━━━━━━━━━━━━ 46s 631ms/step - accuracy: 0.7180 - loss: 0.5142 - val_accuracy: 0.6679 - val_loss: 0.6167
Epoch 5/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 522ms/step - accuracy: 0.7285 - loss: 0.4877

73/73 ━━━━━━━━━━━━━━━━━━━━ 45s 620ms/step - accuracy: 0.7283 - loss: 0.4878 - val_accuracy: 0.6679 - val_loss: 0.6046


In [14]:
model.save("/content/ResNet_spine_model.h5")

In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve
import numpy as np

# Load the saved model
model = load_model('/content/ResNet_spine_model.h5')

# Define test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/Augmented_dataset-1/valid/reorganized',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Predict probabilities
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator).ravel()

# Find optimal threshold based on F1 score
precision, recall, thresholds = precision_recall_curve(y_true, y_pred_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal Threshold (F1-based): {optimal_threshold:.4f}")

# Apply optimal threshold
y_pred = (y_pred_prob > optimal_threshold).astype(int)

# Classification report
print(classification_report(y_true, y_pred))

# ROC-AUC Score
roc_auc = roc_auc_score(y_true, y_pred_prob)
print(f"ROC-AUC Score: {roc_auc:.2f}")


Found 277 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 475ms/step
Optimal Threshold (F1-based): 0.3845
              precision    recall  f1-score   support

           0       0.82      0.70      0.76       185
           1       0.53      0.68      0.60        92

    accuracy                           0.70       277
   macro avg       0.68      0.69      0.68       277
weighted avg       0.72      0.70      0.70       277

ROC-AUC Score: 0.71


##EfficientNetB0

In [2]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

# Define data generators with stronger augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Load train and validation data
train_generator = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = valid_datagen.flow_from_directory(
    '/content/Augmented_dataset-1/valid/reorganized',  # Ensure this points to reorganized folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Load pre-trained EfficientNetB0 without top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)  # Fewer units with L2 regularization
x = Dropout(0.7)(x)  # Higher dropout rate
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),  # More aggressive early stopping
    ModelCheckpoint("best_efficientnet_model.h5", save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks
)

# Unfreeze all layers for fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Recompile with lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training
history_fine = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
    callbacks=callbacks
)

Found 2326 images belonging to 2 classes.
Found 277 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 577ms/step - accuracy: 0.6657 - loss: 1.3995

73/73 ━━━━━━━━━━━━━━━━━━━━ 79s 758ms/step - accuracy: 0.6662 - loss: 1.3953 - val_accuracy: 0.6679 - val_loss: 0.7671
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step - accuracy: 0.7163 - loss: 0.7152

73/73 ━━━━━━━━━━━━━━━━━━━━ 31s 423ms/step - accuracy: 0.7162 - loss: 0.7149 - val_accuracy: 0.6679 - val_loss: 0.6804
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step - accuracy: 0.7263 - loss: 0.6450

...

73/73 ━━━━━━━━━━━━━━━━━━━━ 32s 435ms/step - accuracy: 0.7087 - loss: 0.6330 - val_accuracy: 0.6679 - val_loss: 0.6445
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step - accuracy: 0.7024 - loss: 0.6278

73/73 ━━━━━━━━━━━━━━━━━━━━ 31s 422ms/step - accuracy: 0.7026 - loss: 0.6277 - val_accuracy: 0.6679 - val_loss: 0.6406
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step - accuracy: 0.7246 - loss: 0.6177

73/73 ━━━━━━━━━━━━━━━━━━━━ 32s 442ms/step - accuracy: 0.7244 - loss: 0.6178 - val_accuracy: 0.6679 - val_loss: 0.6396
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step - accuracy: 0.7147 - loss: 0.6185

73/73 ━━━━━━━━━━━━━━━━━━━━ 31s 426ms/step - accuracy: 0.7147 - loss: 0.6185 - val_accuracy: 0.6679 - val_loss: 0.6382
Epoch 9/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 32s 431ms/step - accuracy: 0.7148 - loss: 0.6120 - val_accuracy: 0.6679 - val_loss: 0.6447
Epoch 10/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step - accuracy: 0.7034 - loss: 0.6239

73/73 ━━━━━━━━━━━━━━━━━━━━ 41s 429ms/step - accuracy: 0.7036 - loss: 0.6238 - val_accuracy: 0.6679 - val_loss: 0.6378
Epoch 1/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.7182 - loss: 0.6259 - val_accuracy: 0.6679 - val_loss: 0.6400
Epoch 2/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 34s 468ms/step - accuracy: 0.7195 - loss: 0.6139 - val_accuracy: 0.6679 - val_loss: 0.6490
Epoch 3/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 33s 449ms/step - accuracy: 0.7218 - loss: 0.5955 - val_accuracy: 0.6679 - val_loss: 0.6418
Epoch 4/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 34s 459ms/step - accuracy: 0.7151 - loss: 0.5820 - val_accuracy: 0.6679 - val_loss: 0.6471
Epoch 5/5
73/73 ━━━━━━━━━━━━━━━━━━━━ 34s 468ms/step - accuracy: 0.7188 - loss: 0.5711 - val_accuracy: 0.6679 - val_loss: 0.6443


In [3]:
model.save("/content/EfficientNet_spine_model.h5")

In [5]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve
import numpy as np

# Load the saved model
model = load_model('/content/best_efficientnet_model.h5')

# Define test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/Augmented_dataset-1/valid/reorganized',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Predict probabilities
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator).ravel()

# Find optimal threshold based on F1 score
precision, recall, thresholds = precision_recall_curve(y_true, y_pred_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal Threshold (F1-based): {optimal_threshold:.4f}")

# Apply optimal threshold
y_pred = (y_pred_prob > optimal_threshold).astype(int)

# Classification report
print(classification_report(y_true, y_pred))

# ROC-AUC Score
roc_auc = roc_auc_score(y_true, y_pred_prob)
print(f"ROC-AUC Score: {roc_auc:.2f}")


Found 277 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 783ms/step
Optimal Threshold (F1-based): 0.3235
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       185
           1       0.33      0.99      0.49        92

    accuracy                           0.33       277
   macro avg       0.16      0.49      0.25       277
weighted avg       0.11      0.33      0.16       277

ROC-AUC Score: 0.43
